# ResNets Flower Classification Implement : resnet18 , resnet34 , resnet50 , resnet101 , resnet152

## LƯU Ý : Với Model ResNet này lên đến ~59tr tham số . Rất khó trong việc traning . Mặc khác nếu cho batch size và epoch thấp thì độ chính xác lại không cao . 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os
# os.chdir('/content/drive/MyDrive/DeepLearning/Research/Flower_Classification')

In [ ]:
# file_path = '/content/drive/MyDrive/DeepLearning/Research/Flower_Classification/Dataset/imagelabels.mat' # cách 1
# file_path = './Dataset/imagelabels.mat' # cách 2 , vì ta đã có chỉ định nơi làm việc ở đầu tiên là : os.chdir('/content/drive/MyDrive/DeepLearning/Research/Flower_Classification')
# !unzip cũng nên dùng cách ngày cho ngắn gọn
# !unzip ./Dataset/Dataset_102flower_TOP15.zip -d ./Dataset

In [ ]:
# !unzip /content/drive/MyDrive/DeepLearning/Research/Flower_Classification/Dataset/Dataset_102flower_TOP15.zip -d /content/drive/MyDrive/DeepLearning/Research/Flower_Classification/Dataset

In [ ]:
# !pip install imutils # cách fix trên kaggle 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate, BatchNormalization, Dropout, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from imutils import paths
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report , confusion_matrix
import os
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.io import loadmat

#### Dataset

- Label of Flower : Load file .mat 
- Trong folder 102flowers các file ảnh được sắp xếp theo đúng thứ tự với label trong file .mat 
- Label của 102 loài hoa là bằng số từ 1 đến 102 , thật ra nó không ảnh hưởng gì cả , text hay số đều cũng chỉ là một label mà thôi 
- Chứa 8189 ảnh gồm 102 loài hoa 
- Ta có thể rename lại toàn bộ ảnh bằng các thêm label vào phía trước 

Ta có thể huấn luyện model này bằng dữ liệu 8189 ảnh gồm 102 loài hoa , nó sẽ học để nhận diện đặc điểm các loài hoa , sau khi thu được model này ta sẽ dùng nó để : Transfer learning : feature extractor dùng cho bài toán số nhãn thay đổi . Giả sử ta thêm loài hoa thứ 103 .

Hoặc ta có thể sử dụng dữ liệu 8189 gồm 102 loài hoa này Transfer learning : Fine tuning lại một model nào đó , sau đó dùng nó để Transfer learning : Featre Extractor cũng để dùng cho bài toán số nhãn thay đổi 

Traning bằng TOP 15 loài hoa có số lượng mẫu nhiều nhất 

In [ ]:
# file_path = '/content/drive/MyDrive/DeepLearning/Research/Flower_Classification/Dataset/imagelabels.mat' # cách 1
file_path = './Dataset/imagelabels.mat' # cách 2 , vì ta đã có chỉ định nơi làm việc ở đầu tiên là : os.chdir('/content/drive/MyDrive/DeepLearning/Research/Flower_Classification')
mat_data = loadmat(file_path)
labels = mat_data['labels']
unique_labels, counts = np.unique(labels[0], return_counts=True)
sorted_indices = np.argsort(counts)[::-1]
top_labels = unique_labels[sorted_indices][:15]
top_counts = counts[sorted_indices][:15]
i = 1
total = 0
for label, count in zip(top_labels, top_counts):
    print(f"{i}. {label}: {count}")
    total += count
    i += 1
print('Have',total,'img of 15 label')
labels = top_labels
unique_labels = np.unique(labels).astype(str)
new_unique_labels = []
for label in unique_labels:
    new_label = label + 'label'
    new_unique_labels.append(new_label)
new_unique_labels = np.array(new_unique_labels)

label_names = new_unique_labels
print(label_names)

#### Tiền xử lí dữ liệu

In [ ]:
root_dir = './Dataset/Dataset_102flower_TOP15'
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')

## Datagenerator

In [ ]:
img_size = (224 , 224)
batch_size = 2
img_shape = (img_size[0] , img_size[1] , 3)

# tr_gen = ImageDataGenerator(rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,
                                #    zoom_range=0.2,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

train_gen = tr_gen.flow_from_directory(train_dir , target_size = img_size , class_mode = 'categorical' ,
                                       color_mode = 'rgb' , shuffle = True , batch_size = batch_size)

valid_gen = ts_gen.flow_from_directory(val_dir , target_size = img_size , class_mode = 'categorical' ,
                                       color_mode = 'rgb' , shuffle = True , batch_size = batch_size)

test_gen = ts_gen.flow_from_directory(test_dir , target_size = img_size , class_mode = 'categorical' ,
                                       color_mode = 'rgb' , shuffle = False , batch_size = batch_size)


## Show Sample From Train Data

In [ ]:
g_dict = train_gen.class_indices
classes = list(g_dict.keys())
images, labels = next(train_gen)
num_samples = len(images)

plt.figure(figsize=(20, 20))

for i in range(min(16, num_samples)):
    plt.subplot(4, 4, i + 1)
    image = images[i] / 255
    plt.imshow(image)
    class_index = np.argmax(labels[i]) # Lấy chỉ mục của lớp có xác suất cao nhất
    class_name = classes[class_index]
    plt.title(class_name, color='blue', fontsize=12)
    plt.axis('off')
plt.show()

#### Xây dựng Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import sys

class BasicBlock(tf.keras.Model):
    expansion = 1

    def __init__(self, in_channels, out_channels, strides=1):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()

        if strides != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = tf.keras.Sequential([
                layers.Conv2D(self.expansion*out_channels, kernel_size=1, strides=strides, use_bias=False),
                layers.BatchNormalization()
            ])
        else:
            self.shortcut = lambda x: x

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = layers.add([self.shortcut(x), out])
        out = tf.keras.activations.relu(out)
        return out

class BottleNeck(tf.keras.Model):
    expansion = 4

    def __init__(self, in_channels, out_channels, strides=1):
        super(BottleNeck, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=1, use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.conv3 = layers.Conv2D(self.expansion*out_channels, kernel_size=1, use_bias=False)
        self.bn3 = layers.BatchNormalization()

        if strides != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = tf.keras.Sequential([
                layers.Conv2D(self.expansion*out_channels, kernel_size=1, strides=strides, use_bias=False),
                layers.BatchNormalization()
            ])
        else:
            self.shortcut = lambda x: x

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = tf.keras.activations.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = layers.add([self.shortcut(x), out])
        out = tf.keras.activations.relu(out)
        return out

class BuildResNet(tf.keras.Model):
    def __init__(self, block, num_blocks, num_classes):
        super(BuildResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = layers.Conv2D(64, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.layer1 = self._make_layer(block, 64, num_blocks[0], strides=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], strides=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], strides=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], strides=2)
        self.avg_pool2d = layers.AveragePooling2D(pool_size=4)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool2d(out)
        out = self.flatten(out)
        out = self.fc(out)
        return out

    def _make_layer(self, block, out_channels, num_blocks, strides):
        stride = [strides] + [1]*(num_blocks-1)
        layer = []
        for s in stride:
            layer += [block(self.in_channels, out_channels, s)]
            self.in_channels = out_channels * block.expansion
        return tf.keras.Sequential(layer)

    def get_config(self):
        config = {
          'block': self.block,
          'num_blocks': self.num_blocks,
          'num_classes': self.num_classes
        }
        return config

def ResNet(model_type, num_classes):
    if model_type == 'resnet18':
        return BuildResNet(BasicBlock, [2, 2, 2, 2], num_classes)
    elif model_type == 'resnet34':
        return BuildResNet(BasicBlock, [3, 4, 6, 3], num_classes)
    elif model_type == 'resnet50':
        return BuildResNet(BottleNeck, [3, 4, 6, 3], num_classes)
    elif model_type == 'resnet101':
        return BuildResNet(BottleNeck, [3, 4, 23, 3], num_classes)
    elif model_type == 'resnet152':
        return BuildResNet(BottleNeck, [3, 8, 36, 3], num_classes)
    else:
        sys.exit(ValueError("{:s} is currently not supported.".format(model_type)))

In [ ]:
ResNetModel = ResNet(model_type = 'resnet152', num_classes = len(label_names)) # 15 nhãn 

In [ ]:
ResNetModel.build((1, 224, 224, 3))
ResNetModel.summary()

- Nếu chạy đoạn code load model ở trên google colab thì nó hiện ra được kiến trúc , còn dưới này nó không hiện

#### Fit model

In [ ]:
# Tạo đối tượng optimizer  # SGD
# optimizer = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
# optimizer = tf.keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95)
# optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
# optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.001, learning_rate_power=-0.5, initial_accumulator_value=0.1, l1_regularization_strength=0.0, l2_regularization_strength=0.0)
# optimizer = tf.keras.optimizers.Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999)
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

# Thiết lập ModelCheckpoint
save_best_only = True
checkpoint = ModelCheckpoint('models/Resnet152-model-epoch-{epoch:03d}.keras',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=save_best_only,
                             mode='auto')

# Xây dựng quy trình huấn luyện
ResNetModel.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình với dữ liệu đã chuẩn bị
H = ResNetModel.fit(
    train_gen,
    steps_per_epoch = train_gen.samples // train_gen.batch_size,
    validation_data = valid_gen,
    validation_steps = valid_gen.samples // valid_gen.batch_size,
    epochs = 3,
    callbacks=[checkpoint]
)

## Model Performance

In [ ]:
train_acc = H.history['accuracy']
train_loss = H.history['loss']

val_acc = H.history['val_accuracy']
val_loss = H.history['val_loss']

index_loss = np.argmin(val_loss)
index_acc = np.argmax(val_acc)

val_lowest = val_loss[index_loss]
val_highest = val_acc[index_acc]

Epochs = [i+1 for i in range(len(train_acc))]

loss_label = f'Best Epoch = {str(index_loss + 1)}'
acc_label = f'Best Epoch = {str(index_acc + 1)}'

plt.figure(figsize= (20,8))
plt.style.use('fivethirtyeight')

plt.subplot(1,2,1)
plt.plot(Epochs , train_loss , 'r', label = 'Training Loss')
plt.plot(Epochs , val_loss , 'g' , label = 'Validation Loss')
plt.scatter(index_loss +1 , val_lowest , s = 150 , c = 'blue' , label = loss_label)
plt.title('Training vs Validation (loss)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(Epochs , train_acc , 'r', label= 'Training Accuracy')
plt.plot(Epochs , val_acc , 'g' , label = 'Validation Accuracy')
plt.scatter(index_acc + 1 , val_highest , s= 150 , c = 'blue' , label= acc_label)
plt.title('Training vs Validation (Accuracy)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

## Model Evaluation

In [ ]:
train_score = ResNetModel.evaluate(train_gen , steps= 32 , verbose = 1)
val_score = ResNetModel.evaluate(valid_gen , steps = 32 , verbose = 1)
test_score = ResNetModel.evaluate(test_gen , steps = 32 , verbose = 1)

print(f'Train loss = {train_score[0] }')
print(f'Train Accuracy = {train_score[1]}')
print(f'Validation loss = {val_score[0]}')
print(f'Validation Accuracy = {val_score[1]}')
print(f'Test loss = {test_score[0]}')
print(f'Test Accuracy = {test_score[1]}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Dự đoán
preds = ResNetModel.predict(test_gen)
preds = np.argmax(preds, axis=1)

# Lấy nhãn thật
true_labels = test_gen.classes

# Tính các chỉ số
report = classification_report(true_labels, preds, output_dict=True)

# In chỉ số f1, precision, recall
print("F1:", report['weighted avg']['f1-score'])
print("Precision:", report['weighted avg']['precision'])
print("Recall:", report['weighted avg']['recall'])

In [ ]:
print(classification_report(true_labels, preds))

## Model Prediction

## Confusion Matrix and Classification Report

In [ ]:
# Load model 
from tensorflow.keras.models import load_model
best_model_ConvNeXt = load_model('models/ConvNeXt_model-022.keras')

#### Lấy mẫu ra từ test_gen để dự đoán 

In [ ]:
# Lấy một mẫu ngẫu nhiên từ test_gen
random_index = random.randint(0, len(test_gen) - 1)  
batch = test_gen[random_index] 

# In hình ảnh 
random_img = random.randint(0, len(batch) - 1)  
img = batch[0][random_img]
img = img.astype(float)/255
plt.imshow(img)
plt.axis('off')
plt.show()
# In ra nhãn 
label = batch[1][random_img]
print('label encoder', batch[1][0])
print('Nhãn ban đầu :', classes[np.argmax(label)])


# Chuyển đổi thành 4D tensor
img = batch[0][random_img]
print(img.shape)
img = np.expand_dims(img, axis=0)
print(img.shape)
# Dự đoán
pred = best_model_ConvNeXt.predict(img)
print('Dự đoán:', classes[np.argmax(pred)])

#### Lấy mẫu ảnh ra từ folder test để dự đoán 

In [ ]:
# Lấy một file ảnh ngẫy nhiên trong folder test 
test_folder = 'Dataset/Dataset_Alzheimer_6400/test'
subfolders = [os.path.join(test_folder, subfolder) for subfolder in os.listdir(test_folder) if os.path.isdir(os.path.join(test_folder, subfolder))]
random_subfolder = random.choice(subfolders)
subfolder_images = os.listdir(random_subfolder)
random_image_name = random.choice(subfolder_images)
random_image_path = os.path.join(random_subfolder, random_image_name)

# In ra tên của ảnh được chọn
print("Tên file ảnh được chọn:", random_image_name)

# Load ảnh và chuyển thành tensor
img = load_img(random_image_path)
img = img.resize((32, 32))  # Resize ảnh về kích thước (32, 32)
img_array = img_to_array(img)
img_tensor = tf.expand_dims(img_array, axis=0)  # Thêm chiều batch
# print(img_tensor.shape)
# print(img_tensor)

# Load mô hình đã được lưu trữ
# Dự đoán nhãn của ảnh
predictions = best_model_ConvNeXt.predict(img_tensor)

# In ra nhãn dự đoán
print('Dự đoán:', classes[np.argmax(predictions)])


In [ ]:
# Tính confusion matrix
conf_mat = confusion_matrix(true_labels, preds)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

# Assume you already have the confusion matrix (cm), sorted confusion matrix (sorted_cm), DataFrame (df_cm),
# true labels (true_labels), predicted labels (preds), and classes defined

# Tạo ma trận nhầm lẫn
cm = confusion_matrix(true_labels, preds)

# Sắp xếp lại ma trận nhầm lẫn
sorted_cm = cm[np.argsort(classes)][:, np.argsort(classes)]

# Tạo dataframe từ ma trận nhầm lẫn đã sắp xếp lại
df_cm = pd.DataFrame(sorted_cm, index=classes, columns=classes)

# Set font size
font_size = 10

# Vẽ ma trận nhầm lẫn với font size đã điều chỉnh
plt.figure(figsize=(8, 6))
sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g', annot_kws={"size": font_size})
plt.xlabel('Predicted labels', fontsize=font_size)
plt.ylabel('True labels', fontsize=font_size)
plt.xticks(fontsize=font_size)
plt.yticks(fontsize=font_size)
plt.show()
